In [1]:
import numpy as np
from transformers import CamembertTokenizerFast, CamembertForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset, load_metric

# Create a larger synthetic dataset with more varied examples
data = [
    {"tokens": ["Le", "patient", "a", "de", "la", "fièvre", "et", "des", "maux", "de", "tête", "."],
     "ner_tags": [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]},
    {"tokens": ["Il", "souffre", "de", "douleurs", "abdominales", "et", "de", "nausées", "."],
     "ner_tags": [0, 1, 0, 1, 1, 0, 0, 1, 0]},
    {"tokens": ["La", "patiente", "présente", "une", "toux", "et", "des", "difficultés", "respiratoires", "."],
     "ner_tags": [0, 1, 0, 0, 1, 0, 0, 1, 1, 0]},
    {"tokens": ["Elle", "a", "des", "douleurs", "thoraciques", "et", "une", "fièvre", "élevée", "."],
     "ner_tags": [0, 0, 0, 1, 1, 0, 0, 1, 1, 0]},
    {"tokens": ["Le", "patient", "a", "un", "rhume", "et", "se", "plaind", "de", "maux", "de", "gorge", "."],
     "ner_tags": [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0]},
    {"tokens": ["Elle", "est", "fatiguée", "et", "a", "des", "douleurs", "musculaires", "."],
     "ner_tags": [0, 0, 1, 0, 0, 0, 1, 1, 0]},
    {"tokens": ["Il", "présente", "des", "éruptions", "cutanées", "et", "de", "la", "fièvre", "."],
     "ner_tags": [0, 0, 0, 1, 1, 0, 0, 0, 1, 0]},
    {"tokens": ["Le", "patient", "a", "des", "vertiges", "et", "des", "troubles", "de", "la", "vision", "."],
     "ner_tags": [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]},
    {"tokens": ["Elle", "souffre", "de", "migraine", "et", "a", "des", "nausées", "."],
     "ner_tags": [0, 1, 0, 1, 0, 0, 0, 1, 0]},
    {"tokens": ["Il", "a", "une", "grosse", "fièvre", "et", "se", "plaind", "de", "frissons", "."],
     "ner_tags": [0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0]},
    
    # Additional varied examples
    {"tokens": ["Le", "malade", "a", "des", "courbatures", "et", "de", "la", "fatigue", "."],
     "ner_tags": [0, 1, 0, 0, 1, 0, 0, 0, 1, 0]},
    {"tokens": ["Elle", "ressent", "une", "douleur", "au", "ventre", "et", "a", "perdu", "l'appétit", "."],
     "ner_tags": [0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0]},
    {"tokens": ["Il", "a", "des", "démangeaisons", "et", "une", "éruption", "cutanée", "."],
     "ner_tags": [0, 0, 0, 1, 0, 0, 1, 1, 0]},
    {"tokens": ["Le", "patient", "se", "plaint", "de", "douleurs", "articulaires", "et", "de", "fatigue", "."],
     "ner_tags": [0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0]},
    {"tokens": ["Elle", "a", "des", "difficultés", "à", "respirer", "et", "de", "la", "fièvre", "."],
     "ner_tags": [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0]},
    {"tokens": ["Il", "a", "perdu", "du", "poids", "et", "se", "sent", "faible", "."],
     "ner_tags": [0, 0, 0, 1, 0, 0, 0, 0, 1, 0]},
    {"tokens": ["Le", "patient", "a", "une", "sensation", "de", "brûlure", "à", "la", "miction", "."],
     "ner_tags": [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0]},
    {"tokens": ["Elle", "a", "des", "problèmes", "digestifs", "et", "se", "sent", "ballonnée", "."],
     "ner_tags": [0, 0, 0, 1, 1, 0, 0, 0, 1, 0]},
    {"tokens": ["Il", "se", "plaint", "de", "douleurs", "au", "dos", "et", "de", "raideurs", "musculaires", "."],
     "ner_tags": [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0]},
    {"tokens": ["Elle", "a", "des", "douleurs", "menstruelles", "et", "des", "nausées", "pendant", "ses", "règles", "."],
     "ner_tags": [0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0]}
]

# Convert to Dataset
dataset = Dataset.from_list(data)

# Load tokenizer
tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")

# Define label names
label_names = ["O", "B-ILLNESS"]

# Function to tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True, padding='max_length', max_length=128)
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply tokenization and alignment
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Load model
model = CamembertForTokenClassification.from_pretrained("camembert-base", num_labels=len(label_names))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
)

# Load metric
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [[label_names[p] for (p, l) in zip(prediction, label) if l != -100]
                        for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\saeed\anaconda3\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\saeed\AppData\Local\Temp\ipykernel_38664\1028247650.py:98: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
C:\Users\saeed\anaconda3\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metr

{'eval_loss': 0.366670697927475, 'eval_precision': 0.9814814814814815, 'eval_recall': 0.7361111111111112, 'eval_f1': 0.8412698412698414, 'eval_accuracy': 0.9056603773584906, 'eval_runtime': 2.006, 'eval_samples_per_second': 9.97, 'eval_steps_per_second': 2.493, 'epoch': 5.0}
